# Uploading our stuff, but without 10% loss

    First, lets import our tools & ingredients again

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
df = pd.read_excel('map.xlsx')
print(df.head())

In [ ]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

In [ ]:
def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return name, num

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; j = 0
    
    for i in range(start, end):
        name, num = get_category_subcategory(df, i); num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name))
        except psycopg2.errors.UniqueViolation as e:
            listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
        
        if(j==10):
            con.commit(); j = 0 # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions)); return listOfCollisions

In [ ]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

for i in range(46): # first 46 hundreds - should do it, but 15 mins seems like an issue - but lets buckle up none the less
    collisions = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    print("100 number ", (i+1) ," done")

In [ ]:
cursor.close()
con.close()